In [2]:
import pandas as pd
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF

import pickle

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df=pd.read_pickle("all_recipes.pkl")
df2=pd.read_pickle("all_users.pkl")

,calories,category,ingredients,ratings,recipe_id,reviews,title,total_mins,Season,DaytimeOfCooking,Veg/NonVeg
0,728,"['Desserts', 'Fruit Desserts', 'Pineapple Dess...","['2 cups all-purpose flour', '1 teaspoon salt'...",4.703704,263516,23,Chef John's Carrot Cake,95,Fall/Winter,Dessert,Non-Veg (Eggs)
1,692,"['Bread', 'Yeast Bread']","['1/2 cup warm milk', '1/3 cup warm water', '1...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115,Any,Lunch/Dinner,Non-Veg (Cheese)
2,209,"['Soups, Stews and Chili', 'Stews', 'Beef']","['1 (1 1/2-pound) flank steak', '2 teaspoons k...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690,Any,Dinner,Non-Veg (Beef)
3,505,"['Everyday Cooking', 'Vegetarian', 'Side Dishes']","['2 cups whole milk', '1 teaspoon kosher salt,...",4.333333,272461,9,"""Instant"" Mac and Cheese",28,Any,Lunch/Dinner,Veg
4,291,"['Bread', 'Yeast Bread']","['1 cup warm water', '1 tablespoon white sugar...",5.000000,271049,4,Kouign-Amann,275,Any,Dessert,Non-Veg (Butter)


In [4]:
df = df.dropna()
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   calories          1161 non-null   object 
 1   category          1161 non-null   object 
 2   ingredients       1161 non-null   object 
 3   ratings           1161 non-null   float64
 4   recipe_id         1161 non-null   int64  
 5   reviews           1161 non-null   object 
 6   title             1161 non-null   object 
 7   total_mins        1161 non-null   int64  
 8   Season            1161 non-null   object 
 9   DaytimeOfCooking  1161 non-null   object 
 10  Veg/NonVeg        1161 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 99.9+ KB


In [5]:
df2.dropna()
df2.drop_duplicates(inplace=True)
df2=df2.reset_index(drop=True)

def processing(value):
    return int(value)

df2['rating']=df2['rating'].apply(processing)
df2['recipe_id']=df2['recipe_id'].apply(processing)
df2['user_id']=df2['user_id'].apply(processing)

df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d')

In [6]:
def replace_k(value):
    if 'k' in value:
        value=str(value).split('k')
        value=value[0]
        value=float(value)*1000
    return value

df['reviews']=df['reviews'].apply(replace_k)

def cat_2(value):
    return eval(value)

df['category']=df['category'].apply(cat_2)
df['ingredients']=df['ingredients'].apply(cat_2)

df['reviews']=df['reviews'].apply(processing)

def calories(value):
    if value.isdigit():
        return int(value)
    else:
        return 1
df['calories']=df['calories'].apply(calories)

df = df.rename(columns={'ratings': 'avg_rating', 'reviews':'no_reviews', 'Season':'season','DaytimeOfCooking':'meal_type','Veg/NonVeg':'diet_type'})

df['season'] = df['season'].apply(lambda x: x.split('/'))
df['season'] = df['season'].apply(lambda x: ['Summer', 'Spring', 'Fall', 'Winter'] if x == ['Any'] else x)
df['meal_type'] = df['meal_type'].apply(lambda x: x.split('/'))

def count_ingredients(ingredients):
    return len(ingredients)

df['no_ingredients'] = df['ingredients'].apply(count_ingredients)


,calories,category,ingredients,avg_rating,recipe_id,no_reviews,title,total_mins,season,meal_type,diet_type,no_ingredients
0,728,"[Desserts, Fruit Desserts, Pineapple Desserts]","[2 cups all-purpose flour, 1 teaspoon salt, 1 ...",4.703704,263516,23,Chef John's Carrot Cake,95,"[Fall, Winter]",[Dessert],Non-Veg (Eggs),18
1,692,"[Bread, Yeast Bread]","[1/2 cup warm milk, 1/3 cup warm water, 1 1/2 ...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",Non-Veg (Cheese),14
2,209,"[Soups, Stews and Chili, Stews, Beef]","[1 (1 1/2-pound) flank steak, 2 teaspoons kosh...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690,"[Summer, Spring, Fall, Winter]",[Dinner],Non-Veg (Beef),24
3,505,"[Everyday Cooking, Vegetarian, Side Dishes]","[2 cups whole milk, 1 teaspoon kosher salt, or...",4.333333,272461,9,"""Instant"" Mac and Cheese",28,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",Veg,10
4,291,"[Bread, Yeast Bread]","[1 cup warm water, 1 tablespoon white sugar, 1...",5.000000,271049,4,Kouign-Amann,275,"[Summer, Spring, Fall, Winter]",[Dessert],Non-Veg (Butter),10


In [7]:
pattern = re.compile(r'\s*\([^)]*\)')
records = df['diet_type']

cleaned_records = [re.sub(pattern, '', record) for record in records]
df['diet_type'] = cleaned_records

In [8]:
def clean_text(text):
    text = text.lower()
    nopunct_text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = word_tokenize(nopunct_text)
    return tokens[0]

df['diet_type'] = df['diet_type'].apply(clean_text)

,calories,category,ingredients,avg_rating,recipe_id,no_reviews,title,total_mins,season,meal_type,diet_type,no_ingredients
0,728,"[Desserts, Fruit Desserts, Pineapple Desserts]","[2 cups all-purpose flour, 1 teaspoon salt, 1 ...",4.703704,263516,23,Chef John's Carrot Cake,95,"[Fall, Winter]",[Dessert],nonveg,18
1,692,"[Bread, Yeast Bread]","[1/2 cup warm milk, 1/3 cup warm water, 1 1/2 ...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",nonveg,14
2,209,"[Soups, Stews and Chili, Stews, Beef]","[1 (1 1/2-pound) flank steak, 2 teaspoons kosh...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690,"[Summer, Spring, Fall, Winter]",[Dinner],nonveg,24
3,505,"[Everyday Cooking, Vegetarian, Side Dishes]","[2 cups whole milk, 1 teaspoon kosher salt, or...",4.333333,272461,9,"""Instant"" Mac and Cheese",28,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",veg,10
4,291,"[Bread, Yeast Bread]","[1 cup warm water, 1 tablespoon white sugar, 1...",5.000000,271049,4,Kouign-Amann,275,"[Summer, Spring, Fall, Winter]",[Dessert],nonveg,10


**OUTLIERS REMOVAL USING IQR METHOD**

IQR is a robust statistical measure that represents the spread of the middle 50% of the data. It's calculated as the difference between the third quartile (Q3) and the first quartile (Q1).
In recipe recommendation, outliers could be recipes with:

    Extremely high or low cooking time.
    Extremely high or low calories.
    Unusually large or small numbers of ingredients.

In [9]:
def iqr_method(outlier):
    #Removing Outliers by IQR Method
    q1=df[outlier].quantile(0.25)
    q3=df[outlier].quantile(0.75)
    iqr=q3-q1
    upper_limit = q3 + (1.5*iqr)
    lower_limit = q1 - (1.5*iqr)
    
    df.loc[(df[outlier]>upper_limit),outlier]=upper_limit
    df.loc[(df[outlier]<lower_limit),outlier]=lower_limit
iqr_method('calories')
iqr_method('no_ingredients')
iqr_method('total_mins')

/tmp/ipykernel_79167/2791429900.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '312.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[(df[outlier]>upper_limit),outlier]=upper_limit


**MATRIX FACTORIZATION USING USER RATING AND COMBINATION OF INGREDIENTS,TITLE AND CATEGORY OF FOOD**

MATRIX FACTORIZATION IS COMMONLY USED FOR DETECTING A PATTERN BETWEEN DIFFERENT USERS AND THEIR INTEREST. SINCE WE HAVE DATASET OF USER RATING FOR DIFFERENT RECIPES, WE COULD PLOT USER VS. RECIPE MATRIX. THIS COULD SUGGEST RECIPES FOR A USER THAT WAS ACCEPTED BY A SIMILAR USER.

the parameter on which the user-recipe plotting is done is through the CLICKS, SAVES, and RATED recipes.
we take an interaction parameter which is a combination of these 3 features, with an added weight.
at the initial stage of the website, data on clicks and saves is lacking, hence more weight to rating is given.


FOR THIS MODEL, THE INPUT IS USER_ID AND OUTPUT IS A SERIES OF RECIPE_ID'S


In [11]:
clicks = pd.read_csv('user_clicks.csv')
saved_recipes = pd.read_csv('user_save.csv')
rated_recipes = df2[['recipe_id','user_id','rating']]

# Taking only those recipes that were given a rating of 3 or above. Any rating less than 3 signifies dislike.
rated_recipes = rated_recipes[rated_recipes['rating']>=3]

# Initially the user activity would be null, so less weightage would be given to clicked and saved recipes.
weight_click = 1
weight_saved = 2
weight_rating = 5

interactions = pd.merge(clicks, saved_recipes, on=['user_id', 'recipe_id'], how='outer')
interactions = pd.merge(interactions, rated_recipes, on=['user_id', 'recipe_id'], how='outer')

# Fill missing values with 0
interactions = interactions.fillna(0)

# Apply weights and create the unified interaction score
interactions['interaction'] = (weight_click * interactions['click'] + weight_saved * interactions['saved'] + weight_rating * interactions['rating'])

user_item_matrix = interactions.pivot_table(values='interaction', index='user_id', columns='recipe_id').fillna(0)
print(user_item_matrix)

recipe_id  219075  219076  219077  219078  219079  219082  219086  219088  \
user_id                                                                     
16            0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
39            0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
338           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
935           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1005          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...           ...     ...     ...     ...     ...     ...     ...     ...   
25586309      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
25591130      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
25594700      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
25595047      0.0     0.0     0.0     0.0    25.0     0.0     0.0     0.0   
25598991      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

In [15]:
num_factors=10

#perform NMF Non-Negative Matrix Factorization
nmf = NMF(n_components=num_factors, random_state=42)
user_factors=nmf.fit_transform(user_item_matrix)
recipe_factors=nmf.components_

user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_item_matrix.index)}

# Function to recommend recipes based on user ID
def recommend_recipes(user_id, top_n=10):
    if user_id not in user_id_to_index:
        print(f"User ID {user_id} not found.")
        return []
    
    user_index = user_id_to_index[user_id]
    user_factor = user_factors[user_index]
    predicted_ratings = user_factor.dot(recipe_factors)
    top_recipe_indices = predicted_ratings.argsort()[-top_n:][::-1]  # Get top n indices, sorted in descending order
    
    # Map indices back to recipe IDs
    top_recipe_ids = user_item_matrix.columns[top_recipe_indices]
    return top_recipe_ids

#recommendations for user with ID 
recommended_recipes_CF=recommend_recipes(16)

print(recommended_recipes_CF)

Index([219166, 234534, 220127, 219077, 233170, 219967, 239047, 221887, 222187,
       234803],
      dtype='int64', name='recipe_id')


In [16]:
pickle.dump(recommend_recipes, open('CF_model_renewed.pkl','wb'))
CF_recommend_recipes = pickle.load(open("CF_model_renewed.pkl", "rb"))
CF_recommend_recipes(16)

Index([219166, 234534, 220127, 219077, 233170, 219967, 239047, 221887, 222187,
       234803],
      dtype='int64', name='recipe_id')